### **환경 설정**

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:46tcmalloc: large alloc 1147494400 bytes == 0x396dc000 @  0x7f26ba9d9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:11:34tcmalloc: large alloc 1434370048 bytes == 0x7dd32000 @  0x7f26ba9d9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.3 MB/s eta 0:08:55tcmalloc: large alloc 1792966656 bytes =

In [3]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.1
11.1
GCC 7.3


In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### **데이터셋 다운로드**

In [6]:
# Download dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MY_API_KEY")
project = rf.workspace("sketch").project("sketch-detection")
dataset = project.version(2).download("coco")

     |████████████████████████████████| 145 kB 8.9 MB/s 
     |████████████████████████████████| 178 kB 64.6 MB/s 
     |████████████████████████████████| 1.1 MB 70.6 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 138 kB 79.2 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=f4829f1de26712ad1a924f868c9dc0e1e01864e14da6dc49f4ef83fcae7b3b8e
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=734f1092c4b4fc17ea2f561431bab8c21f1ade01f72a435715b059beea0ea81a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to sketch-detection-2 in coco:: 100%|██████████| 3309/3309 [00:02<00:00, 1249.00it/s]


### **YoloX**

<img src="https://user-images.githubusercontent.com/40661020/144001736-9fb303dd-eac7-46b0-ad45-214cfa51e928.png" width=500/>

In [10]:
from mmcv import Config
from mmdet.apis import set_random_seed

cfg = Config.fromfile('./configs/yolox/yolox_s_8x8_300e_coco.py')

print(cfg.pretty_text)

optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=300)
checkpoint_config = dict(interval=10)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_met

In [11]:
# config 설정

data_root = '/content/mmdetection/sketch-detection-2/'
ann_file = '_annotations.coco.json'


cfg.data_root = data_root

cfg.train_dataset.dataset.ann_file = data_root + 'train/' + ann_file
cfg.train_dataset.dataset.img_prefix = data_root + 'train/'
cfg.train_dataset.dataset.classes = ('button', 'checkbox_checked', 'checkbox_unchecked',
                            'chip', 'data_table', 'dropdown_menu',
                            'floating_action_button', 'image', 'radiobutton_checked',
                            'radiobutton_unchecked', 'slider', 'switch_disabled',
                            'switch_enabled', 'text_field', 'text_area')

cfg.data.train = cfg.train_dataset

cfg.data.val.ann_file = data_root + 'valid/' + ann_file
cfg.data.val.img_prefix = data_root + 'valid/'
cfg.data.val.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.test.ann_file = data_root + 'test/' + ann_file
cfg.data.test.img_prefix = data_root + 'test/'
cfg.data.test.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.optimizer.lr = 0.0025

cfg.model.bbox_head.num_classes = 15

cfg.runner.max_epochs = 30
cfg.max_epochs = 30

cfg.work_dir = 'working/yolox'

cfg.seed = 0

cfg.gpu_ids = [0]
cfg.device = 'cuda'

cfg.log_config.interval = 100
cfg.checkpoint_config.interval = 1
cfg.evaluation.interval = 1
cfg.evaluation.metric = 'bbox'


In [18]:
print(cfg.pretty_text)

# config 파일 저장
with open("yolox.py","w") as f:
  f.write(cfg.pretty_text)

!cp yolox.py /content/drive/MyDrive/model/yolox


optimizer = dict(
    type='SGD',
    lr=0.0025,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=30)
checkpoint_config = dict(interval=1)
log_config = dict(interval=100, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_me

In [ ]:
# 학습

import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Save checkpoints
!cp -r /content/mmdetection/working/yolox /content/drive/MyDrive/model

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-05-14 11:34:01,472 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-14 11:34:01,498 - mmdet - INFO - Start running, host: root@41d76338b268, work_dir: /content/mmdetection/working/yolox
2022-05-14 11:34:01,499 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) YOLOXLrUpdaterHook                 
(49          ) ExpMomentumEMAHook                 
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) YOLOXLrUpdaterHook                 
(48          ) YOLOXModeSwitchHook                
(48          ) SyncNormHook                       
(49          ) ExpMomentumEMAHook                 
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 ------------------

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-05-14 11:34:21,177 - mmdet - INFO - Epoch [1][100/1352]	lr: 5.471e-07, eta: 2:12:25, time: 0.196, data_time: 0.036, memory: 1721, loss_cls: 10.8492, loss_bbox: 4.9840, loss_obj: 1772.6003, loss: 1788.4335
2022-05-14 11:34:38,008 - mmdet - INFO - Epoch [1][200/1352]	lr: 2.188e-06, eta: 2:02:39, time: 0.168, data_time: 0.012, memory: 1838, loss_cls: 10.8499, loss_bbox: 4.9848, loss_obj: 758.0179, loss: 773.8526
2022-05-14 11:34:54,949 - mmdet - INFO - Epoch [1][300/1352]	lr: 4.924e-06, eta: 1:59:27, time: 0.169, data_time: 0.012, memory: 1838, loss_cls: 10.6735, loss_bbox: 4.9722, loss_obj: 243.5916, loss: 259.2373
2022-05-14 11:35:12,057 - mmdet - INFO - Epoch [1][400/1352]	lr: 8.753e-06, eta: 1:57:59, time: 0.171, data_time: 0.012, memory: 1838, loss_cls: 10.1434, loss_bbox: 4.9825, loss_obj: 75.5889, loss: 90.7148
2022-05-14 11:35:28,949 - mmdet - INFO - Epoch [1][500/1352]	lr: 1.368e-05, eta: 1:56:42, time: 0.169, data_time: 0.012, memory: 1838, loss_cls: 9.8919, loss_bbox: 4.98

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 33.9 task/s, elapsed: 9s, ETA:     0s

...

2022-05-14 13:15:16,554 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-05-14 13:15:16,898 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.823
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.823
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.823
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.823
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.958
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.958
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.958
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.10s).


2022-05-14 13:15:17,385 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_30.pth.
2022-05-14 13:15:17,398 - mmdet - INFO - Best bbox_mAP is 0.8230 at 30 epoch.
2022-05-14 13:15:17,413 - mmdet - INFO - Epoch(val) [30][298]	bbox_mAP: 0.8230, bbox_mAP_50: 0.8230, bbox_mAP_75: 0.8230, bbox_mAP_s: -1.0000, bbox_mAP_m: -1.0000, bbox_mAP_l: 0.8230, bbox_mAP_copypaste: 0.823 0.823 0.823 -1.000 -1.000 0.823
